<a href="https://colab.research.google.com/github/prd-dahal/Transfer-Learning---Nepali-Currency-Detection/blob/master/Currency_Detection_using_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
print(os.listdir('/content/gdrive/My Drive/MONEY DETECTION DATA/valid'))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
['100', '10', '50', '500', '1000', '20', 'None', '5']


In [1]:
#dependencies  
import keras 
from keras import Model, Sequential, layers, models
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from keras import regularizers

Using TensorFlow backend.


In [0]:
#variable declerations 
image_width = 256
image_height = 256
batch_size = 16
no_of_training_sample = 250
no_of_validation_sample = 50
epochs = 100
train_path = '/content/gdrive/My Drive/MONEY DETECTION DATA/train'
valid_path = '/content/gdrive/My Drive/MONEY DETECTION DATA/valid'


In [5]:
#preparing model 
model = keras.applications.VGG19(include_top=False, weights='imagenet',input_shape=(image_height,image_width,3))
#freezing all the layers except  last layer 
for layer in model.layers[:-2]:
  layer.trainable = False

#adding custom layers
x = model.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01))(x)
predictions = layers.Dense(8, activation='softmax')(x)
model_final = Model(inputs=model.input, outputs=predictions)
model_final.compile(loss = 'categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model_final.summary()

80142336/80134624 [==============================] - 3s 0us/step
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
____________________________________________________________

In [8]:

ImageFile.LOAD_TRUNCATED_IMAGES = True
#data augumentation in train set
train_data_generator = ImageDataGenerator(
    horizontal_flip=True,
    rescale = 1./255,
    fill_mode ='nearest',
    zoom_range = 0.3,
    width_shift_range=0.3,
    height_shift_range=0.3,
    rotation_range=30

)
#data augumentation in valid test
valid_data_generator = ImageDataGenerator(
    horizontal_flip =True,
    rescale = 1./255,
    fill_mode='nearest',
    zoom_range = 0.3,
    width_shift_range = 0.3,
    rotation_range = 30,
    height_shift_range = 0.3
)

#specifying train data directory and resize
train_generator = train_data_generator.flow_from_directory(
    train_path,
    target_size = (image_height, image_width),
    class_mode = 'categorical'
)

#specifying test data directory and resize 
valid_generator = valid_data_generator.flow_from_directory(
    valid_path,
    target_size = (image_height, image_width),
    class_mode = 'categorical'
)


Found 2000 images belonging to 8 classes.
Found 560 images belonging to 8 classes.


In [6]:
#fit the model 

model_final.fit_generator(train_generator,
                          samples_per_epoch=no_of_training_sample, 
                          epochs= epochs, 
                          validation_data=valid_generator,
                          
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=100, validation_data=<keras.pre..., steps_per_epoch=7)`
  """


Epoch 1/100
7/7 [==============================] - 542s 77s/step - loss: 25.9862 - accuracy: 0.1429 - val_loss: 16.4548 - val_accuracy: 0.1429
Epoch 2/100
7/7 [==============================] - 266s 38s/step - loss: 13.9690 - accuracy: 0.1827 - val_loss: 11.7050 - val_accuracy: 0.2714
Epoch 3/100
7/7 [==============================] - 276s 39s/step - loss: 10.6929 - accuracy: 0.3571 - val_loss: 9.7999 - val_accuracy: 0.3071
Epoch 4/100
7/7 [==============================] - 270s 39s/step - loss: 8.7692 - accuracy: 0.4330 - val_loss: 7.5802 - val_accuracy: 0.3125
Epoch 5/100
7/7 [==============================] - 271s 39s/step - loss: 7.0670 - accuracy: 0.4420 - val_loss: 6.2354 - val_accuracy: 0.3696
Epoch 6/100
7/7 [==============================] - 269s 38s/step - loss: 5.7258 - accuracy: 0.5179 - val_loss: 5.2780 - val_accuracy: 0.4804
Epoch 7/100
7/7 [==============================] - 276s 39s/step - loss: 4.9809 - accuracy: 0.5714 - val_loss: 4.9270 - val_accuracy: 0.4982
Epoch 8/

In [0]:
model_final.save('money_detection.h5')

(2340, 4160, 3)


array([[2.780876e-23, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 1.000000e+00, 5.911976e-10, 0.000000e+00]],
      dtype=float32)

In [19]:
import os
import matplotlib.pyplot as plt
import cv2
import keras

# path = '/content/gdrive/My Drive/MONEY DETECTION DATA/valid/20'
# os.listdir(path)
image_path = '/content/gdrive/My Drive/MONEY DETECTION DATA/train/100/Compressed_IMG_20181009_133724.jpg'
image = cv2.imread(image_path)
print(image.shape)
plt.imshow(image)

dim = (256,256)
pred_image = cv2.resize(image,dim,interpolation = cv2.INTER_AREA)
pred_image = pred_image.reshape(1,256,256,3)

predicted = model_final.predict(pred_image)
predicted
#map = {0:10, 1:5, 2:500, 3:20, 4:5, 5:50, 6:500, 7:'None'}